In [ ]:
#Simulador TP 6

import random
import math
import numpy as np
import logging

#GENERAR ALEATORIAS
def valorRandom():
  r=random(0,1)
  return r

#EVENTOS
def llegadaFamiliar():
  global T, TF
  global MB, N ,M
  global TPSI, TPSF
  global TPLLB, TPSB, TPSBC
  global NG, NI, NSBC, NTBC
  global TPLLF,TPLLI
  global NT, NTA, NTB,NTG
  global PPS , SPS
  global PTAB, PTOA, PTOB, PTOBC
  global STOA, STOB, STOBC
  global CONTAB
  global ITOBC, ITOB
  SPSF+=NG*(TPLLF-T)
  SPSI+=NI*(TPLLF-T)
  T=TPLLF
  #TPLLF=T+IAF
  r=valorRandom
  if(NG-M>5 & r>0.3):
    ARRF+=1
  else:
    NG+=1
    NTG+=1
    if(NM<M):
      NM+=1
      #busca en las mesas la que tenga tps=HV
      indiceMesaDisponible=buscarMesaDisponble(TPSF) 
      if(NM<=3*G):
        #genera un tiempo de atencion TA
        TA=generarTiempoDeAtencion()
        #cambia el valor de tps de la mesa a T+TA
        TPSF[indiceMesaDisponible]=T+TA
      else:
        ITOF[indiceMesaDisponible]=T

def salidaFamiliar():
  #busco la mesa a salir
  indiceMesaSalida=buscarMesaProximaASalir(TPSF)

  SPSF+=NG*(TPSF[indiceMesaSalida]-T)
  SPSI+=NI*(TPSF[indiceMesaSalida]-T)
  T=TPSF[indiceMesaSalida]
  TPSF[indiceMesaSalida]=HV
  NG-=1
  NM-=1
  if(NM>=3*G):
    #busca en las mesas la que tenga tps=HV
    indiceMesaDisponible=buscarMesaDisponble(TPSF)
    #termino el tiempo ocioso y lo sumo
    STOF+=T-ITO[indiceMesaDisponible]
    ITOF[indiceMesaDisponible]=HV
    #genera un tiempo de atencion TA
    TA=generarTiempoDeAtencion()
    #cambia el valor de tps de la mesa a T+TA
    TPSF[indiceMesaDisponible]=T+TA
  elif(NG>=M):
    NM+=1
    if(NM<=3*G):
      #genera un tiempo de atencion TA
      TA=generarTiempoDeAtencion()
      #cambia el valor de tps de la mesa a T+TA
      TPSF[indiceMesaDisponible]=T+TA
    else:
      ITOF[indiceMesaDisponible]=T

def llegadaNoFamiliar():
  global T, TF
  global MB, N ,M
  global TPSI, TPSF
  global TPLLB, TPSB, TPSBC
  global NG, NI, NSBC, NTBC
  global TPLLF,TPLLI
  global NT, NTA, NTB,NTG
  global PPS , SPSF
  global PTAB, PTOA, PTOB, PTOBC
  global STOA, STOB, STOBC
  global CONTAB
  global ITOBC, ITOB
  SPSF+=NG*(TPLLI-T)
  SPSI+=NI*(TPLLI-T)
  T=TPLLI
  #TPLLI=T+IANF
  r=valorRandom
  if(NG-M>5 & r>0.3):
    ARRI+=1
  else:
    NI+=1
    NTI+=1
    if(NN<N):
      NN+=1
      #busca en las mesas la que tenga tps=HV
      indiceMesaDisponible=buscarMesaDisponble(TPSI) 
      if(NN<=3*I):
        #genera un tiempo de atencion TA
        TA=generarTiempoDeAtencion()
        #cambia el valor de tps de la mesa a T+TA
        TPSI[indiceMesaDisponible]=T+TA
      else:
        ITOI[indiceMesaDisponible]=T

def salidaNoFamiliar():
  #busco la mesa a salir
  indiceMesaSalida=buscarMesaProximaASalir(TPSI)

  SPSF+=NG*(TPSI[indiceMesaSalida]-T)
  SPSI+=NI*(TPSI[indiceMesaSalida]-T)
  T=TPSI[indiceMesaSalida]
  TPSI[indiceMesaSalida]=HV
  NI-=1
  NN-=1
  if(NN>=3*I):
    #busca en las mesas la que tenga tps=HV
    indiceMesaDisponible=buscarMesaDisponble(TPSI)
    #termino el tiempo ocioso y lo sumo
    STOF+=T-ITOI[indiceMesaDisponible]
    ITOI[indiceMesaDisponible]=HV
    #genera un tiempo de atencion TA
    TA=generarTiempoDeAtencion()
    #cambia el valor de tps de la mesa a T+TA
    TPSI[indiceMesaDisponible]=T+TA
  elif(NI>=N):
    NM+=1
    if(NN<=3*I):
      #genera un tiempo de atencion TA
      TA=generarTiempoDeAtencion()
      #cambia el valor de tps de la mesa a T+TA
      TPSI[indiceMesaDisponible]=T+TA
    else:
      ITOI[indiceMesaDisponible]=T


#FUNCIONES
def buscarMesaDisponble(tps):
  mesa=tps.index(HV)
  return mesa

def buscarMesaProximaASalir(tps):
  mesa=tps.index(min(mesa))
  return mesa

def obtener_Proximo_Evento():
  #Buscar el tiempo de salida de la prox mesa familiar que se va a desocupar
  indiceFamiliar=buscarMesaProximaASalir(TPSF)
  mesaFamiliarASalir=TPSF[indiceFamiliar]
  
  #Buscar el tiempo de salida de la prox mesa no familiar que se va a desocupar
  indiceNoFamiliar=buscarMesaProximaASalir(TPSI)
  mesaNoFamiliarASalir=TPSF[indiceNoFamiliar]

  eventos=[TPLLF,TPLLI,mesaFamiliarASalir,mesaNoFamiliarASalir]
  return eventos.index(min(eventos))



In [ ]:
def calcularResultados():
    global T, TF #Tiempo Y Tiempo final
    global TPLLF,TPLLI, TPSF,TPSI #Tiempos de llegada y salida
    global NTI,NTF,NI,NN, NM, NG #Cantidades totales de c/cliente, cantiades de c/cliente en el sist. y cantidades de c/mesas usadas
    global PPSF, SPSF,SPSI,PPSI #Permanecia en el sist de c/tipo y sus sumatorias
    global ITOF, ITOI, PTOI, PTOF #Porcentajes de tiempo ocioso de c/cliente (los q se sientan en la mesa sin poder pedir) y los inicios de tiempo ocioso
    global STOI, STOF #Sumatorias de tiempo ocioso
    global N, I, M, G #Variables de control
    global HV ,ARRF,ARRI #High value y cantidad de arrepentidos por tipo de cliente
    
    
    print(f"SIMULACION PARA: \n {N} CANTIDAD DE MESAS FAMILIARES \n {M} CANTIDAD DE MESAS NO FAMILIARES \n {G} CANTIDAD DE MOZOS FAMILIARES \n {i} CANTIDAD DE MOZOS NO FAMILIARES")
    

In [ ]:
#SIMULADOR
def simulacion(cantN,cantM,cantI,cantG):
  global T, TF #Tiempo Y Tiempo final
  global TPLLF,TPLLI, TPSF,TPSI #Tiempos de llegada y salida
  global NTI,NTF,NI,NN, NM, NG #Cantidades totales de c/cliente, cantiades de c/cliente en el sist. y cantidades de c/mesas usadas
  global PPSF, SPSF,SPSI,PPSI #Permanecia en el sist de c/tipo y sus sumatorias
  global ITOF, ITOI, PTOI, PTOF #Porcentajes de tiempo ocioso de c/cliente (los q se sientan en la mesa sin poder pedir) y los inicios de tiempo ocioso
  global STOI, STOF #Sumatorias de tiempo ocioso
  global N, I, M, G #Variables de control
  global HV ,ARRF,ARRI #High value y cantidad de arrepentidos por tipo de cliente

  


  #Sumatoria de permanencia en el sistema
  SPSF=0     
  SPSI=0
  #Arrepentidos
  ARRF=0 
  ARRI=0
  #High value
  HV=9999999999

  # Condiciones Iniciales

  T=0   #tiempo en minutos
  TF= 960  #tiempo final en minutos
  TPLLF=0 #Tiempo de llegada de prox. cliente familiar
  TPLLI=0 #Tiempo de llegada de prox. cliente no familiar
  TPSF=[HV] * MA  #Tiempo de Salida de MESAS FAMILIARES
  TPSI=[HV] * (MB-N)  #Tiempo de salida de MESAS INDIVIDUALES

  #Variables de Control
  N = cantN #cantidad de mesas no familiares
  M = cantM 
  I= cantI #cantidad de mozos no familiares
  G= cantG


  #Variables de Estado

  NN=0   #Cantidad de mesas individuales en uso
  NM=0   #Cantidad de mesas familiares en uso
  NG=0  #Cantidad clientes familiares en el sistema
  NI=0  #Cantidad clientes no familiares en el sistema
  NTI=0 #Cantidad clientes no familiares en total
  NTF=0 #Cantidad clientes familiares en total

  #Variables de Resultados
  PPSF=0   #promedio de permanencia en el sistema de los clientes familiares
  PPSI=0   #promedio de permanencia en el sistema de los clientes no familiares
  PTOI=0
  PTOF=0
  
  SPSI=0
  SPSF=0

  STOI=0
  STOF=0

  ITOF=[T]*M
  ITOI=[T]*N

  print("Inicio: ", minutos_a_dias_horas_minutos(T))
  print("Final: ",minutos_a_dias_horas_minutos(TF))

  #print("STOA: ", STOA)
  #print("ITOA: ", ITOA)

  while True:
    proximo_evento=obtener_Proximo_Evento()
    cases={
            0:llegadaFamiliar,
            1:llegadaNoFamiliar,
            2:salidaFamiliar,
            3:salidaNoFamiliar,
          }
    cases.get(proximo_evento)()

    if (T<=TF):
      continue
    elif(NSA+NSB+NSBC==0):
      break
    else:
      TPLLA=HV
      TPLLB=HV
      continue
    calcularResultados()
    cantN=cantN-1